In [51]:
import os
from tensorflow.keras import layers,models
import tensorflow as tf
import librosa


In [52]:
import os
caputch = os.path.join("C:/Users/hp/Documents/Machine learning/Deep learning/Projects(N)/Deep Audio classification/dataset/Parsed_Capuchinbird_Clips")
not_caputch = os.path.join("C:/Users/hp/Documents/Machine learning/Deep learning/Projects(N)/Deep Audio classification/dataset/Parsed_Not_Capuchinbird_Clips")


In [53]:
#dataset creation
import tensorflow as tf
pt = tf.data.Dataset.list_files(caputch +'/*.wav', shuffle=False)# List all .wav files in the directories
nt = tf.data.Dataset.list_files(not_caputch +'/*.wav', shuffle=False)
pos_labels = tf.data.Dataset.from_tensor_slices(tf.ones(len(list(pt))))# Create labels (1 for positive, 0 for negative)
neg_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(list(nt))))
pos = tf.data.Dataset.zip((pt, pos_labels))# Zip the datasets with their corresponding labels
neg = tf.data.Dataset.zip((nt, neg_labels))
df = pos.concatenate(neg)# Concatenate the positive and negative datasets



In [54]:
# def load_wav_16k_mono(filename):
#     file_contents = tf.io.read_file(filename)
#     wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
#     wav = tf.squeeze(wav, axis=-1)
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     wav = tf.io.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
#     return wav


def load_wav_16k_mono(filename):
    # Convert Tensor to a numpy string to be used with librosa
    filename = filename.numpy().decode('utf-8')  # Ensure the filename is a string
    wav, sample_rate = librosa.load(filename, sr=16000, mono=True)
    wav = tf.convert_to_tensor(wav, dtype=tf.float32)
    return wav



In [62]:
# def preprocess(file_path, label):
#     wav = load_wav_16k_mono(file_path)# Load the .wav file at a 16 kHz sample rate and convert it to mono
#     wav = wav[:48000] # Trim or slice the waveform to ensure it is at most 48000 samples (3 seconds of audio at 16 kHz)
#     zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)    # Create zero-padding to ensure that the waveform is exactly 48000 samples if it is shorter
#     wav = tf.concat([zero_padding, wav], 0)# Concatenate the zero-padding with the waveform to make it exactly 48000 samples long
#     # Apply Short-Time Fourier Transform (STFT) to the waveform to convert it into a spectrogram
#     # frame_length: Size of the window for each FFT (320 samples, ~20 ms at 16 kHz)
#     # frame_step: Number of samples to step between successive frames (32 samples, ~2 ms at 16 kHz)
#     spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
#     spectrogram = tf.abs(spectrogram)# Convert the complex STFT output to its magnitude (absolute values) to represent the power spectrum
#     spectrogram = tf.expand_dims(spectrogram, axis=2)# Expand dimensions to add a channel dimension for compatibility with CNN models (e.g., shape [time, frequency, channels])
#     return spectrogram, label# Return the processed spectrogram and the corresponding label



def preprocess(file_path, label):
    # Use tf.py_function to load the .wav file, wrapping the function call
    wav = tf.py_function(load_wav_16k_mono, [file_path], Tout=tf.float32)# Note: we now pass the file_path as a list
    wav.set_shape([None]) # Set the shape of the wav tensor to allow for variable length
    wav = wav[:48000]  # Trim or slice the waveform to ensure it is at most 48000 samples
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)  # Create zero-padding
    wav = tf.concat([zero_padding, wav], 0)  # Concatenate padding and waveform
    # Apply STFT to waveform to create spectrogram
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)  # Magnitude of STFT
    spectrogram = tf.expand_dims(spectrogram, axis=2)  # Add channel dimension
    return spectrogram, label  # Return the processed spectrogram and the corresponding label



In [63]:
data = df.map(preprocess)
data = data.shuffle(1000).cache().prefetch(buffer_size=tf.data.AUTOTUNE).batch(16)

In [64]:
data

<_BatchDataset element_spec=(TensorSpec(shape=(None, None, 257, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [65]:
len(data)*0.7,len(data)*0.2,len(data)*0.1

(35.699999999999996, 10.200000000000001, 5.1000000000000005)

In [66]:
train = data.take(36)
test = data.skip(36).take(10)
val = data.skip(46).take(5)

In [67]:
samples,labels = train.as_numpy_iterator().next()

In [31]:
samples.shape

(16, 1491, 257, 1)

In [83]:
model = models.Sequential([
    layers.Input(shape=(1491, 257, 1)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Reshape((-1, 64)),  # Reshape to (None, height*width, channels), example: (None, 184*30, 64)
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [84]:
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_36 (Conv2D)              │ (None, 1489, 255, 16)  │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_35 (MaxPooling2D) │ (None, 744, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 742, 125, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 371, 62, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 369, 60, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 184, 30, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 5520, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 353280)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 128)            │    45,219,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,243,393 (172.59 MB)

 Trainable params: 45,243,393 (172.59 MB)

 Non-trainable params: 0 (0.00 B)

In [85]:
sound = model.fit(train,epochs=5,validation_data=val)

Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - loss: 0.8305 - precision: 0.7051 - recall: 0.8147 - val_loss: 0.0576 - val_precision: 0.9615 - val_recall: 1.0000
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - loss: 0.0683 - precision: 0.9646 - recall: 0.9795 - val_loss: 0.0566 - val_precision: 0.9615 - val_recall: 1.0000
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - loss: 0.0095 - precision: 0.9989 - recall: 0.9979 - val_loss: 0.0629 - val_precision: 0.9615 - val_recall: 1.0000
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - loss: 0.0029 - precision: 1.0000 - recall: 0.9979 - val_loss: 0.0644 - val_precision: 0.9615 - val_recall: 1.0000
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - loss: 0.0027 - precision: 1.0000 - recall: 0.9979 - val_loss: 0.0588 - val_precision: 0.9615 - val_recall: 1.0000


In [94]:
X_test, y_test = test.as_numpy_iterator().next()

In [95]:
result = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [98]:
result = [1 if prediction > 0.5 else 0 for prediction in result]

In [99]:
result

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

for mp3

In [ ]:
# def load_mp3_16k_mono(filename):
#     """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
#     res = tfio.audio.AudioIOTensor(filename)
#     # Convert to tensor and combine channels 
#     tensor = res.to_tensor()
#     tensor = tf.math.reduce_sum(tensor, axis=1) / 2 
#     # Extract sample rate and cast
#     sample_rate = res.rate
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     # Resample to 16 kHz
#     wav = tfio.audio.resample(tensor, rate_in=sample_rate, rate_out=16000)
#     return wav